### Ragas is a framework that helps you evaluate your Retrieval Augmented Generation (RAG) pipelines


In [1]:
import os, json


from utils import OPENAI_API_KEY

os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY 
#from agentic_chunker import AgenticChunker
from llm_utils import *
from llm_utils import _text
from tqdm.notebook import tqdm

#os.environ["LANGCHAIN_TRACING_V2"] = "true"

#openai.api_key = os.environ['OPENAI_API_KEY']

[nltk_data] Downloading package punkt_tab to /home/oleg/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/oleg/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


In [2]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser

from langchain_core.pydantic_v1 import BaseModel

#llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)  

In [3]:
#loader_url =SeleniumURLLoader(["https://arxiv.org/pdf/2312.10997",
pdf_list = ["./data/data_rag/s41524-023-01062-z.pdf",
    "./data/data_rag/s41699-023-00369-1.pdf"
         #   "https://github.com/HSE-LAMBDA/ai4material_design/tree/main/docs/CONSTRUCTOR-MOCK.md"
         #   "https://github.com/HSE-LAMBDA/ai4material_design/blob/main/docs/CONSTRUCTOR.md",
         #   "https://github.com/HSE-LAMBDA/ai4material_design/blob/main/docs/DATA.md",
         #   "https://github.com/HSE-LAMBDA/ai4material_design/blob/main/docs/ENVIRONMENT.md",
         #   "https://github.com/HSE-LAMBDA/ai4material_design/blob/main/docs/GENERATING-CONSTRUCTOR.md",
         #   "https://github.com/HSE-LAMBDA/ai4material_design/blob/main/docs/GENERATING-MOCK.md",
         #   "https://github.com/HSE-LAMBDA/ai4material_design/blob/main/docs/PILOT.md",
         #   "https://github.com/HSE-LAMBDA/ai4material_design/blob/main/docs/SPARSE-PAPER.md"
          #  "https://www.nature.com/articles/s41377-024-01407-3",
          #  "https://www.nature.com/articles/s41565-023-01407-1",
          #  "https://www.nature.com/articles/s41699-023-00369-1",
           ]
                               
chunks, documents = load_pdf_documets(pdf_list, tokens=1000)
print(len(chunks), len(documents))

Created a chunk of size 1135, which is longer than the specified 1000


128 35


### Load test QA  dataset

In [4]:
import pandas as pd

df_ground_true = pd.read_csv("./data/data_rag/QA_ai4mat_2articles.csv")

questions = df_ground_true['question'].values.tolist()
answers = df_ground_true['answer'].values.tolist()


### Embeedding of chunks

### Compare 2 stratigies 
#### Recursive chunking,  Multi Hypothetical Queations

In [5]:
from ragas.metrics import (
    context_utilization,
    context_entity_recall,
    context_relevancy,
    answer_relevancy,
    answer_correctness, 
    faithfulness,
    context_precision,
    context_recall,
)

from datasets import Dataset 

from ragas.evaluation import evaluate
import nest_asyncio

In [6]:
metrics = [
   answer_relevancy,
    answer_correctness, 
   faithfulness,
    context_precision,
    context_recall,
 #   context_entity_recall,
    context_relevancy,
 #   context_utilization,
]

In [7]:
emb, q_list, ch_list, dataset, result = dict(), dict(), dict(), dict(), dict()

### Make vectores DB

In [8]:
from tqdm import tqdm
from pymilvus import MilvusClient

milvus_client = MilvusClient("./milvus_demo.db")

#### Recursive chunking

In [9]:
name = 'recursive'
emb[name], q_list[name], ch_list[name] = get_embs(chunks, embedding=OpenAIEmbeddings(),  question='no') 

len(emb[name]), len(emb[name][0]), len(ch_list[name])==len(emb[name])

The same chunks:   0%|          | 0/128 [00:00<?, ?it/s]

(128, 1536, True)

In [10]:
collection_name = f"ai4mat_OpenAiEmb_{name}"

if milvus_client.has_collection(collection_name):
    milvus_client.drop_collection(collection_name)

milvus_client.create_collection(
    collection_name=collection_name,
    dimension=len(emb[name][0]),    # emb dimension
    metric_type="IP",            # Inner product distance
    consistency_level="Strong",  # Strong consistency level
    )
print("dimension",len(emb[name][0]))

dimension 1536


In [11]:
data = []

for i, line in enumerate(tqdm(ch_list[name], desc="Creating embeddings")):
    data.append({"id": i, "vector": emb[name][i], "text": _text(ch_list[name][i]), })



milvus_client.insert(collection_name=collection_name, data=data)

Creating embeddings: 100%|█████████████████████████| 128/128 [00:00<00:00, 164734.86it/s]


{'insert_count': 128,
 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127],
 'cost': 0}

In [12]:
dataset[name] =  get_datasamples(questions, 
                            answers, 
                            milvus_client, 
                            collection_name, 
                            embedding=lambda q: OpenAIEmbeddings().embed_query(q), 
                            llm = llm,
                            top_k=10)


0it [00:00, ?it/s]

In [13]:
nest_asyncio.apply()

#query_engine_bge = build_query_engine(hf_bge, documents)
result[name] = evaluate(dataset[name], metrics, llm=llm,  raise_exceptions=False, embeddings=OpenAIEmbeddings())

Evaluating:   0%|          | 0/210 [00:00<?, ?it/s]

No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
Failed to parse output. Returning None.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
Failed to parse output. Returning None.
No statements were generated from the answer.
No statements were generated from the answer.


#### Multi Hypothetical Queations for Documents

In [14]:
name = 'questions'
emb[name], q_list[name], ch_list[name] = get_embs(documents, embedding=OpenAIEmbeddings(),  question='split') 

len(emb[name]), len(emb[name][0]), len(ch_list[name])==len(emb[name])

Creating questions & split:   0%|          | 0/35 [00:00<?, ?it/s]

(281, 1536, True)

In [15]:
collection_name = f"ai4mat_OpenAiEmb_{name}"

if milvus_client.has_collection(collection_name):
    milvus_client.drop_collection(collection_name)

milvus_client.create_collection(
    collection_name=collection_name,
    dimension=len(emb[name][0]),    # emb dimension
    metric_type="IP",            # Inner product distance
    consistency_level="Strong",  # Strong consistency level
    )
print("dimension",len(emb[name][0]))

dimension 1536


In [16]:
data = []

for i, line in enumerate(tqdm(ch_list[name], desc="Creating embeddings")):
    data.append({"id": i, "vector": emb[name][i], "text": _text(ch_list[name][i]), })

milvus_client.insert(collection_name=collection_name, data=data)

Creating embeddings: 100%|█████████████████████████| 281/281 [00:00<00:00, 176199.64it/s]


{'insert_count': 281,
 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215,

In [17]:
dataset[name] =  get_datasamples(questions, 
                            answers, 
                            milvus_client, 
                            collection_name, 
                            embedding=lambda q: OpenAIEmbeddings().embed_query(q), 
                            llm = llm,
                            top_k=10)


0it [00:00, ?it/s]

In [18]:
nest_asyncio.apply()

#query_engine_bge = build_query_engine(hf_bge, documents)
result[name] = evaluate(dataset[name], metrics, llm=llm,  raise_exceptions=False, embeddings=OpenAIEmbeddings())

Evaluating:   0%|          | 0/210 [00:00<?, ?it/s]

No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from 

#### Multi Hypothetical Queations for Chunks

In [19]:
name = 'ch_questions'
emb[name], q_list[name], ch_list[name] = get_embs(chunks, embedding=OpenAIEmbeddings(),  question='chunk') 

len(emb[name]), len(emb[name][0]), len(ch_list[name])==len(emb[name])

Creating questions:   0%|          | 0/128 [00:00<?, ?it/s]

(930, 1536, True)

In [20]:
collection_name = f"ai4mat_OpenAiEmb_{name}"

if milvus_client.has_collection(collection_name):
    milvus_client.drop_collection(collection_name)

milvus_client.create_collection(
    collection_name=collection_name,
    dimension=len(emb[name][0]),    # emb dimension
    metric_type="IP",            # Inner product distance
    consistency_level="Strong",  # Strong consistency level
    )
print("dimension",len(emb[name][0]))

dimension 1536


In [21]:
data = []

for i, line in enumerate(tqdm(ch_list[name], desc="Creating embeddings")):
    data.append({"id": i, "vector": emb[name][i], "text": _text(ch_list[name][i]), })

milvus_client.insert(collection_name=collection_name, data=data)

Creating embeddings: 100%|██████████████████████████| 930/930 [00:00<00:00, 32260.67it/s]


{'insert_count': 930,
 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215,

In [22]:
dataset[name] =  get_datasamples(questions, 
                            answers, 
                            milvus_client, 
                            collection_name, 
                            embedding=lambda q: OpenAIEmbeddings().embed_query(q), 
                            llm = llm,
                            top_k=10)


0it [00:00, ?it/s]

In [23]:
nest_asyncio.apply()

#query_engine_bge = build_query_engine(hf_bge, documents)
result[name] = evaluate(dataset[name], metrics, llm=llm,  raise_exceptions=False, embeddings=OpenAIEmbeddings())

Evaluating:   0%|          | 0/210 [00:00<?, ?it/s]

No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from 

### Results

In [24]:
def print_dict(d, indent=0):
    """
    Recursively prints a dictionary, including nested dictionaries, 
    with indentation to represent structure.
    """
    for key, value in d.items():
        print('  ' * indent + str(key) + ": ", end="")
        if isinstance(value, dict):
            print()  # Print a newline for nested dictionaries
            print_dict(value, indent + 1)
        else:
            print(str(value))

In [25]:
print_dict(result)

recursive: 
  answer_relevancy: 0.2251211414806174
  answer_correctness: 0.25021229760513114
  faithfulness: 1.0
  context_precision: 0.476958931701444
  context_recall: 0.5095238095238095
  context_relevancy: 0.012575242305061787
questions: 
  answer_relevancy: 0.25052256382601473
  answer_correctness: 0.29252062765876996
  faithfulness: 0.9875
  context_precision: 0.4810856143822227
  context_recall: 0.44682539682539685
  context_relevancy: 0.08773145212595819
ch_questions: 
  answer_relevancy: 0.22492653933293516
  answer_correctness: 0.28768316701284585
  faithfulness: 1.0
  context_precision: 0.5184657659191172
  context_recall: 0.43697478991596633
  context_relevancy: 0.012016894449205463


In [26]:
df_m = pd.DataFrame().from_dict(result, orient='index')

df_m

,answer_relevancy,answer_correctness,faithfulness,context_precision,context_recall,context_relevancy
recursive,0.225121,0.250212,1.0000,0.476959,0.509524,0.012575
questions,0.250523,0.292521,0.9875,0.481086,0.446825,0.087731
ch_questions,0.224927,0.287683,1.0000,0.518466,0.436975,0.012017


In [ ]:
df_m.to_csv('df_compare_multi_questions.csv')

In [ ]:
pd.DataFrame(dataset['questions'])

In [ ]:
pd.DataFrame(dataset['recursive'])

In [ ]:
pd.DataFrame(dataset['ch_questions'])